In [41]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Libraries

Import needed common libraries

In [42]:
import numpy as np
import pandas as pd
import seaborn as sns

# Data Load

Read data. Between The Jakarta Composite Index and BBRI in one year

In [43]:
dataset = pd.read_csv('../input/datasetjksebbri2022/datasets-jkse-bbri-032022.csv')
dataset.head()

In [44]:
print(dataset.shape) 

This means that our dataset has 13 rows and 1 columns.

**Statistical Dataset**

In [45]:
dataset.describe()

**Display Data**

In [46]:
display(dataset.info())
display(dataset.head())

# Constanta and Coefficient

In [47]:
X = dataset['JKSE'].values
Y = dataset['BBRI'].values

In [ ]:
#Mean X and Y
mean_x = np.mean(X)
mean_y = np.mean(Y)

#Total number of values
total_values = len(X)

#using formula to calculate B1 and B2
# coefficient (b0) 
# constanta (b1)
numeric = 0
denom = 0 

for i in range(total_values):
    numeric += (X[i]- mean_x) * (Y[i] - mean_y)
    denom += (X[i] - mean_x)** 2
    
b1 = numeric/denom
b0 = mean_y -(b1 * mean_x)
print(b0,b1)

# 2D Graph

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (20.0 , 10.0)

In [ ]:
#ploting values and regression line
max_x = np.max(X) + 100
min_x = np.min(X) - 100

#calculating line values x and y
x = np.linspace(min_x, max_x, 100)
y = b0+b1*x

#ploting line
plt.plot(x,y, color='#58b970', label = 'Regression Line')

#ploting scatters points
plt.scatter(X,Y, c='#ef5423' , label ='Scatter Plot')

plt.xlabel('JKSE')
plt.ylabel('BBRI')
plt.legend()
plt.show()

In [ ]:
dataset.plot(x='JKSE', y='BBRI', style='o')
plt.title('Hours vs Percentage')
plt.xlabel('Hours Studied')
plt.ylabel('Percentage Score')
plt.show()

From the graph above, we can see the influence of JKSE on the market from BBRI